In [ ]:
from brfinance import CVMAsyncBackend
import pandas as pd
from datetime import date

cvm_httpclient = CVMAsyncBackend()

# Dict de códigos CVM para todas as empresas
cvm_codes = cvm_httpclient.get_cvm_codes()

# Dict de todas as categorias de busca disponíveis (Fato relevante, DFP, ITR, etc.)
categories = cvm_httpclient.get_consulta_externa_cvm_categories()

# Realizando busca por Empresa
ano = int(input("Qual o ano inicial para a pesquisa? "))

start_date = date(ano, 12, 31)
end_dt = date.today()
cvm_codes_list = ['21067'] # Moura_Dubeux
category = ["EST_4"] # Códigos de categoria para DFP, ITR e fatos relevantes #, "EST_3", "IPE_4_-1_-1"
last_ref_date = False # Se "True" retorna apenas o último report no intervalo de datas

#Lista vazia para armazenas os dataframes
dfs = []

# Busca
search_result = cvm_httpclient.get_consulta_externa_cvm_results(
    cod_cvm=cvm_codes_list,
    start_date=start_date,
    end_date=end_dt,
    last_ref_date=last_ref_date,
    category=category
    )

# Filtrar dataframe de busca para DFP e Status Ativo
search_result = search_result[
    (search_result['categoria']=="DFP - Demonstrações Financeiras Padronizadas")]
search_result = search_result[search_result['status']=="Ativo"]
search_result = search_result[search_result['ref_date']>=str(start_date)]
search_result = search_result[pd.to_numeric(search_result['numero_seq_documento'], errors='coerce').notnull()]


reports_list = [
    'Balanço Patrimonial Ativo',
    'Balanço Patrimonial Passivo',
    'Demonstração do Resultado',
    'Demonstração do Fluxo de Caixa'] # Se None retorna todos os demonstrativos disponíveis.

# Obter demonstrativos
for index, row in search_result.iterrows():
    empresa = f"{row['cod_cvm']} - {cvm_codes[row['cod_cvm']]}"

    reports = cvm_httpclient.get_report(row["numero_seq_documento"], row["codigo_tipo_instituicao"], reports_list=reports_list)
    for report in reports:
        reports[report]["cod_cvm"] = row["cod_cvm"]

        #Adicionar os dataframes em uma lista
        dfs.append(reports[report])    

#Concatenar os dataframes em um único dataframe
df_concat = pd.concat(dfs,ignore_index=True)

# Salvar o dataframe no arquivo excel, repetindo as linhas das DFPs conforme os anos do filtro
df_concat.to_excel('Dados_CVM.xlsx', index=False)

# Criar pivot table
pivot_table = pd.pivot_table(df_concat, index=["Conta", "Descrição"], values=["Valor"], aggfunc=lambda x: "\n".join(str(i) for i in x))

# Salvar pivot table em um arquivo Excel
pivot_table.to_excel("Dados_CVM_Filtrados.xlsx")

# Carregar o arquivo pivot_table.xlsx em um DataFrame pandas
df_pivot = pd.read_excel('Dados_CVM_Filtrados.xlsx')

# Obter o número máximo de valores em uma única célula "Valor" em todo o DataFrame
max_values = int(df_pivot['Valor'].str.count('\n').max()) + 1
value_cols = [f'{ano+i}' for i in reversed(range(max_values))]
df_pivot[value_cols] = df_pivot['Valor'].str.split('\n', expand=True)

# Separar os valores da coluna "Valor" em colunas separadas
df_pivot[value_cols] = df_pivot['Valor'].str.split('\n', expand=True)

# Remover a coluna "Valor" original
df_pivot = df_pivot.drop(columns=['Valor'])

# Ordenar colunas em ordem crescente, mantendo as duas primeiras colunas
all_cols = list(df_pivot.columns)
ordered_cols = all_cols[:2] + sorted(all_cols[2:])

# Reagrupamento das colunas no DataFrame
df_pivot = df_pivot.reindex(columns=ordered_cols)

#Filtrar dados
df_pivot.query('Conta == "3.01" | Conta == "3.02" | Conta == "6.01.01.03" \
    | Conta == "3.06" | Conta == "3.08" | Conta == "3.04" | Conta == "3.11" \
    | Conta == "6.01.02" | Conta == "6.02.04"', inplace = True)

#Formatar estilo
df_pivot.style.format(decimal=",")

# Salvar o DataFrame atualizado em um arquivo Excel
df_pivot.to_excel('Case_Base.xlsx', index=False)


In [67]:
############################################# RECEITA LÍQUIDA #########################################################

# Ler o arquivo "Case_Base.xlsx" em um DataFrame
df = pd.read_excel("Case_Base.xlsx")

#Inicializando 
Lucro_Bruto = []
for i in range((len(df.columns)-2)):
    Lucro_Bruto.append(pd.to_numeric(df.loc[df["Conta"].isin(["3.01", "3.02"]), f"{ano+i}"]).sum())

# Criar novo DataFrame com a nova linha
new_row = pd.DataFrame([["3.01+3.02", "Lucro Bruto"] + Lucro_Bruto], columns=df.columns)

# Concatenar o novo DataFrame com o DataFrame original
df = pd.concat([df, new_row], ignore_index=True)

# Salvar o DataFrame atualizado em um arquivo Excel
#df.to_excel("Case_Base_SemEBIT.xlsx", index=False)


In [68]:
############################################# EBIT #########################################################

# selecionando a linha Depreciação e Armotização e multiplicando os valores por -1
#for i in range((len(df.columns)-2)):
    #df.loc[6,f"{ano+i}"] = df.loc[6,f"{ano+i}"] * -1

#Inicializando 
EBIT = []
for i in range((len(df.columns)-2)):
    df.loc[6,f"{ano+i}"] = df.loc[6,f"{ano+i}"] * -1 #selecionando a linha Depreciação e Armotização e multiplicando os valores por -1
    EBIT.append(pd.to_numeric(df.loc[df["Conta"].isin(["3.01+3.02", "3.04", "6.01.01.03"]), f"{ano+i}"]).sum())

# Criar novo DataFrame com a nova linha
new_row = pd.DataFrame([["fx1", "EBIT"] + EBIT], columns=df.columns)

# Concatenar o novo DataFrame com o DataFrame original
df = pd.concat([df, new_row], ignore_index=True)


# Salvar o DataFrame atualizado em um arquivo Excel
#df.to_excel("Case_Base.xlsx", index=False)

In [69]:
############################################# NOPAT #########################################################
#Inicializando 
NOPAT = []
for i in range((len(df.columns)-2)):
    NOPAT.append(pd.to_numeric(df.loc[10,f"{ano+i}"] * (1 - (df.loc[4,f"{ano+i}"]/df.loc[0,f"{ano+i}"]))))

# Criar novo DataFrame com a nova linha
new_row = pd.DataFrame([["fx2", "NOPAT"] + NOPAT], columns=df.columns)

# Concatenar o novo DataFrame com o DataFrame original
df = pd.concat([df, new_row], ignore_index=True)

# Salvar o DataFrame atualizado em um arquivo Excel
#df.to_excel("Case_Base.xlsx", index=False)

In [70]:
############################################# FreeCashFlowtoFirm #########################################################
#Inicializando 
FreeCash = []
for i in range((len(df.columns)-2)):
    FreeCash.append(pd.to_numeric(df.loc[11,f"{ano+i}"] + df.loc[6,f"{ano+i}"] - df.loc[7,f"{ano+i}"] - df.loc[8,f"{ano+i}"]))

# Criar novo DataFrame com a nova linha
new_row = pd.DataFrame([["fx3", "FreeCashFlow"] + FreeCash], columns=df.columns)

# Concatenar o novo DataFrame com o DataFrame original
df = pd.concat([df, new_row], ignore_index=True)

# Salvar o DataFrame atualizado em um arquivo Excel
df.to_excel("Case_Base.xlsx", index=False)